In [ ]:
import cv2 as cv  # Importa la librería OpenCV con el alias cv
import numpy as np  # Importa la librería NumPy con el alias np
import os  # Importa el módulo os para interactuar con el sistema operativo

# Directorio donde se encuentran los datos de entrenamiento
dataSet = "E:/bd/emociones"
# Lista de nombres de carpetas dentro del directorio dataSet, cada una correspondiendo a una categoría/emoción
faces = os.listdir(dataSet)
print(faces)

labels = []  # Lista para almacenar las etiquetas de las imágenes
facesData = []  # Lista para almacenar las imágenes
label = 0  # Inicializa la etiqueta

# Itera sobre cada carpeta (cada categoría/emoción)
for face in faces:
    # Ruta de la carpeta que contiene las imágenes de esta categoría/emoción
    facePath = dataSet+'/'+face
    # Itera sobre cada imagen dentro de la carpeta
    for faceName in os.listdir(facePath):
        # Agrega la etiqueta correspondiente a la lista de etiquetas
        labels.append(label)
        # Lee la imagen en escala de grises y la agrega a la lista de datos de las imágenes
        facesData.append(cv.imread(facePath+'/'+faceName, 0))
    # Incrementa la etiqueta para la siguiente categoría/emoción
    label = label + 1

# Crea un objeto reconocedor de rostros utilizando el algoritmo LBPH
faceRecognizer = cv.face.LBPHFaceRecognizer_create()
# Entrena el reconocedor de rostros con los datos de las imágenes y sus etiquetas
faceRecognizer.train(facesData, np.array(labels))
# Guarda el modelo entrenado en un archivo XML
faceRecognizer.write('emocionesv0LBPHFace.xml')

In [ ]:
import cv2 as cv  # Importa la librería OpenCV con el alias cv
import os  # Importa el módulo os para interactuar con el sistema operativo

def detectar_rostros():
    # Directorio donde se encuentran los datos de entrenamiento
    dataSet = "E:/bd/emociones"
    # Lista de nombres de carpetas dentro del directorio dataSet, cada una correspondiendo a una categoría/emoción
    faces = os.listdir(dataSet)
    print(faces)

    # Crea un objeto reconocedor de rostros utilizando el algoritmo LBPH y lo carga desde el archivo XML previamente entrenado
    faceRecognizer = cv.face.LBPHFaceRecognizer_create()
    faceRecognizer.read('emocionesv0LBPHFace.xml')

    # Inicia la captura de video desde la cámara
    cap = cv.VideoCapture(0)
    # Carga el clasificador de Haar para detectar rostros
    rostro = cv.CascadeClassifier('haarcascade_frontalface_alt.xml')
    while True:
        # Lee un fotograma de la cámara
        ret, frame = cap.read()
        # Si no se puede leer el fotograma, sale del bucle
        if ret == False:
            break
        # Convierte el fotograma a escala de grises
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        # Realiza una copia del fotograma en escala de grises
        cpGray = gray.copy()
        # Detecta rostros en el fotograma
        rostros = rostro.detectMultiScale(gray, 1.3, 3)
        # Itera sobre cada rostro detectado
        for(x, y, w, h) in rostros:
            # Extrae la región de interés (ROI) que contiene el rostro
            frame2 = cpGray[y:y+h, x:x+w]
            # Cambia la escala del ROI a 100x100 píxeles
            frame2 = cv.resize(frame2,  (100,100), interpolation=cv.INTER_CUBIC)
            # Realiza la predicción del rostro utilizando el reconocedor de rostros
            result = faceRecognizer.predict(frame2)
            # Dibuja un texto en el fotograma con el resultado de la predicción
            cv.putText(frame, '{}'.format(result), (x,y-20), 1,3.3, (255,255,0), 1, cv.LINE_AA)
            # Si la confianza de la predicción es menor a 70
            if result[1] < 9500:
                # Dibuja un texto en el fotograma con la categoría/emoción identificada
                cv.putText(frame,'{}'.format(faces[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv.LINE_AA)
                # Dibuja un rectángulo alrededor del rostro identificado
                cv.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
            else:
                # Si la confianza de la predicción es mayor o igual a 70, se considera como 'Desconocido'
                cv.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv.LINE_AA)
                # Dibuja un rectángulo alrededor del rostro no identificado
                cv.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)
        # Muestra el fotograma con los rostros detectados
        cv.imshow('frame', frame)
        # Espera por una tecla presionada durante 1 milisegundo
        k = cv.waitKey(1)
        # Si se presiona la tecla Esc (código 27 en OpenCV), sale del bucle
        if k == 27:
            break
    # Libera los recursos de la cámara y cierra todas las ventanas
    cap.release()
    cv.destroyAllWindows()

# Llama a la función detectar_rostros para iniciar el proceso de detección de rostros
detectar_rostros()